In [1]:
import sys
sys.path.insert(0, '/Users/kvolk/Documents/GitHub/SBDynT/src')
import horizons_api

In [2]:
horizons_api.query_sb_from_jpl(des='54598',clones=2)

13.17932074511404 0.2020525393502441
best fit:  16.516526958281602 0.2020525393502441 0.3622083579312695 5.89729170396269 2.6625372979296915 -1.2465025357667279
clones  16.51653029227247 0.20205206772936393 0.3622084329443451 5.89729154372402 2.662536859867709 -1.2465028206457656
clones  16.516530346574704 0.20205278617840616 0.362208227762196 5.897291517691193 2.662536866402187 -1.246501501738115


(1,
 2456908.5,
 array([12.7252781 , 12.72527986, 12.72527883]),
 array([8.60454094, 8.6045435 , 8.60454078]),
 array([4.83563461, 4.83563738, 4.83563337]),
 array([-0.18782377, -0.18782368, -0.18782381]),
 array([0.16547946, 0.16547951, 0.16547945]),
 array([0.03130538, 0.03130541, 0.03130535]))